In [1]:
import json
import javalang
from tqdm import tqdm
import collections

# 从源文件中，将camel_code, source_code, comments 分离到三个文件中
## camel_code是后续网络训练的code输入
## source_code处理后得到AST,进而得到sbt序列
## comments 是目标真实值 

In [2]:
dataset_path='./dataset.json'

camel_code_path='./camel_code.txt'

source_code_path='./source_code.txt'

comment_path='./comment.txt'

In [3]:
def getCode_Nl(a,c,d,e):
    # a: 源文件
    # c: camel_code
    # d: source_code
    # e: comment
    with open(a,'r',encoding='utf-8') as testdata:
        camel_code=open(c,'w+',encoding='utf-8')
        source_code=open(d,'w+',encoding='utf-8')
        nl=open(e,'w+',encoding='utf-8')
        for line in tqdm(testdata.readlines()):
            line=json.loads(line)
            
            
            camel_code.write(line['camel_code'])
            camel_code.write('\n')
           
            tokens=list(javalang.tokenizer.tokenize(line['source_code']))
            newsource=""
            for tk in tokens:
                if tk.__class__.__name__=="String" or tk.__class__.__name__=="Character":
                    newsource=newsource+" "+"STR_"
                elif "Integer" in tk.__class__.__name__ or "FloatingPoint" in tk.__class__.__name__:
                    newsource=newsource+" "+"NUM_"
                elif tk.__class__.__name__=="Boolean":
                    newsource=newsource+" "+"BOOL_"
                else:
                    newsource=newsource+" "+tk.value
            newsource=newsource.strip()
        
            source_code.write(newsource)
            source_code.write('\n')
            
            nl.write(line['nl'])
            nl.write('\n')
            
        camel_code.close()
        source_code.close()
        nl.close()

In [4]:
getCode_Nl(dataset_path,camel_code_path,source_code_path,comment_path)

100%|████████████████████████████████████████████████████| 485812/485812 [02:32<00:00, 3189.09it/s]


## 对source_code进行处理，得到AST

In [5]:
ast_path='./ast.json'

In [6]:
def getast(source,target):
    with open(source,'r',encoding='utf-8') as sfile:
        lines=sfile.readlines()
    with open(target,'w+',encoding='utf-8') as tfile:
        count=0
        ign_count=0
        for line in tqdm(lines):
            line=line.strip()
            
            tokens=javalang.tokenizer.tokenize(line)
            tokenslist=list(javalang.tokenizer.tokenize(line))
            length=len(tokenslist)
            
            parser=javalang.parser.Parser(tokens)
            
            try:
                tree=parser.parse_member_declaration()
                
            except(javalang.parser.JavaSyntaxError,IndexError,StopIteration,TypeError):
                #print(line)
                #pass
                count+=1
                continue
            
            flatten=[]
            for path,node in tree:
                # flatten[0]的type是啥？
                flatten.append({'path':path,'node':node})
                
            # 不符合ast的标志
            ign=False
            # ast中间存储
            outputs=[]
            
            
            # 下面for循环得到子树关系
            for i,Node in enumerate(flatten):
                d=collections.OrderedDict()
                path=Node['path']
                node=Node['node']
                
                children=[]
                # for 循环得到树的结构信息
                for child in node.children:
                    child_path=None
                    # 如果child是ast的节点类型
                    if isinstance(child,javalang.ast.Node):
                        # tuple((node,))代表啥?
                        child_path=path+tuple((node,))
                        for j in range(i+1,len(flatten)):
                            if child_path==flatten[j]['path'] and child==flatten[j]['node']:
                                children.append(j)
                    # 如果child是list类型，并且不为空
                    if isinstance(child,list) and child:
                        child_path=path+(node,child)
                        for j in range(i+1,len(flatten)):
                            if child_path==flatten[j]['path']:
                                children.append(j)
                d['id']=i
                d['type']=str(node.__class__.__name__)
                #d['type']=str(node)
                #if children:
                d['children']=children
                value=None
                # 下面得到终端节点（叶节点）value信息
                if hasattr(node,'name'):
                    value=node.name
                elif hasattr(node,'value'):
                    value=node.value
                elif hasattr(node,'position') and node.position:
                    for i,token in enumerate(tokenslist):
                        if node.position == token.position:
                            pos=i+1
                            value=str(token.value)
                            # a . b ==> a.b
                            while(pos<length and tokenslist[pos].value=='.'):
                                value=value+'.'+tokenslist[pos+1].value
                                # 有必要pos+2吗?有:a . b . c
                                pos+=2
                            break
                elif type(node) is javalang.tree.This or type(node) is javalang.tree.ExplicitConstructorInvocation:
                    value='this'
                elif type(node) is javalang.tree.BreakStatement:
                    value='break'
                elif type(node) is javalang.tree.ContinueStatement:
                    value='continue'
                elif type(node) is javalang.tree.TypeArgument:
                    value=str(node.pattern_type)
                elif type(node) is javalang.tree.SuperMethodInvocation or type(node) is javalang.tree.SuperMemberReference:
                    value='super.'+str(node.member)
                elif type(node) is javalang.tree.Statement or type(node) is javalang.tree.BlockStatement or type(node) is javalang.tree.ForControl or type(node) is javalang.tree.ArrayInitializer or type(node) is javalang.tree.SwitchStatementCase:
                    value='None'
                elif type(node) is javalang.tree.VoidClassReference:
                    value='void.class'
                elif type(node) is javalang.tree.SuperConstructorInvocation:
                    value='super'
                    
                if value is not None and type(value) is type('str'):
                    d['value']=value
                # 无子树，也无值。无子树代表是终端节点，终端节点必有值。这种ast树是不正确的
                if not children and not value:
                    #print(type(node))
                    #print(line)
                    ign=True
                    ign_count+=1
                    break
                outputs.append(d)
            if not ign:
                tfile.write(json.dumps(outputs))
                tfile.write('\n')
        print("语法错误",count)
        print(ign_count)

In [7]:
getast(source_code_path,ast_path)

100%|█████████████████████████████████████████████████████| 485812/485812 [13:13<00:00, 612.40it/s]

语法错误 0
0


## 对AST进行遍历，得到SBT序列 

In [8]:
sbt_path='./sbt.txt'

In [9]:
def get_sbt_process(line,r):
    sbt=''
    #sbt=line[r]['children']
    if not line[r]['children'] :#or not hasattr(line[r],'children')
        #if not hasattr(line[r],'children'):
        #    print(1)
        sbt=" ( "+line[r]['type']+" ) "+line[r]['type']
    else:
        sbt=" ( "+line[r]['type']
        for i in line[r]['children']:
            sbt=sbt+get_sbt_process(line,i)
        sbt=sbt+" ) "+line[r]['type']
    return sbt

In [10]:
def getsbt(a,b):
    with open(a,'r',encoding='utf-8') as ast:
        lines=ast.readlines()
    with open(b,'w',encoding='utf-8') as sbt:
        for line in tqdm(lines):
            line=json.loads(line)
            #print(type(line))
            #for i in range(len(line)):
             #   print(line[i])
            #print(type(line[0]))
            sbtcode=get_sbt_process(line,0)
            sbtcode=sbtcode.strip()
            sbt.write(sbtcode+'\n')
            #print(sbt)
            #print(type(line[0]['children']))
            #break

In [11]:
getsbt(ast_path,sbt_path)

100%|███████████████████████████████████████████████████| 485812/485812 [00:22<00:00, 21985.44it/s]
